In [2]:
import importlib
import funciones_clusters as cl
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import re
import pandas as pd
import seaborn as sns
import copy
from mne.viz import plot_evoked_topo
from mne.channels import find_ch_adjacency, make_1020_channel_selections
from mne.stats import spatio_temporal_cluster_test, ttest_ind_no_p, spatio_temporal_cluster_1samp_test
from functools import partial
%matplotlib qt5

In [16]:
importlib.reload(cl)

<module 'funciones_clusters' from 'c:\\Users\\jhquiza\\OneDrive - Universidad de Medellin\\JupyterNotebooks\\IAT\\funciones_clusters.py'>

In [4]:
# Diccionario con evocados de todos los sujetos, para ensayos congruentes e incongruentes
rutas_archivos = []
ruta_dir = os.getcwd() + '\\Sin filtrar\\'
dir = os.listdir(ruta_dir)
for q in range(len(dir)):
    ruta_arc =ruta_dir + dir[q]
    rutas_archivos.append(ruta_arc)

# Diccionario con evocados de todos los sujetos, para ensayos congruentes e incongruentes
evocados_co = {}
evocados_in = {}
for l in range(len(rutas_archivos)):
    ruta=rutas_archivos[l]
    epochs = mne.read_epochs(ruta)
    epochs.filter(l_freq=None, h_freq=30)
    evoked_co = epochs['12','13','14','15','16','17','18','19'].average()
    evocados_co[ruta[-14:-9]] = evoked_co
    evoked_in = epochs['24','25','26','27','28','29','30','31'].average()
    evocados_in[ruta[-14:-9]] = evoked_in

Reading c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21100n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
145 matching events found
No baseline correction applied
0 projection items activated
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 113 samples (0.441 sec)

Reading c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21101n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available

In [39]:
# Configuro tamaños de fuentes
params = {'axes.labelsize':20, 'axes.titlesize': 20, 'legend.fontsize':20, 'legend.loc': 'upper left', 'xtick.labelsize': 15, 'ytick.labelsize': 15}
plt.rcParams.update(params)

# Análisis entre grupos de sujetos

In [5]:
ambos = ['23001', '23003', '23008', '23022', '23025']
exguerrilleros = ['21100', '21106', '21107', '21108', '21110', '21111', '21113', '21114', '21115', '21116', '21118', '21121', 
                  '21122', '21125', '21131', '21137', '21140', '21143', '21144', '21147', '23007', '23012']
exparamilitares = ['21101', '21102', '21103', '21104', '21105', '21109', '21112', '21123', '21127', '21129', '21132', '21134', 
                   '21135', '21139', '21152', '23002', '23004', '23005', '23006', '23009', '23010', '23011', '23013', '23015',
                   '23016', '23017', '23018', '23019', '23020', '23021', '23023']
victimas = ['24002', '24003', '24004', '24005', '24013', '24014', '24015', '24017', '24025', '24026', '24027', '24035', 
            '24039', '24043', '24045', '24052', '24053', '24057', '24059', '24065', '24069', '24073', '24101']
controles = ['22100', '22101', '22102', '22103', '22104', '22106', '22107', '22108', '22110', '22111', '22112', '22113', 
             '22114', '22115', '22116', '22117']

In [10]:
ga_ambos_co, ga_ambos_in, ambos_co, ambos_in, tiempos, canales, adjacency = cl.matrices_cluster(ambos, evocados_co, evocados_in)
ga_exguerrilleros_co, ga_exguerrilleros_in, exguerrilleros_co, exguerrilleros_in, tiempos, canales, adjacency = cl.matrices_cluster(exguerrilleros, evocados_co, evocados_in)
ga_exparamilitares_co, ga_exparamilitares_in, exparamilitares_co, exparamilitares_in, tiempos, canales, adjacency = cl.matrices_cluster(exparamilitares, evocados_co, evocados_in)
ga_victimas_co, ga_victimas_in, victimas_co, victimas_in, tiempos, canales, adjacency = cl.matrices_cluster(victimas, evocados_co, evocados_in)
ga_controles_co, ga_controles_in, controles_co, controles_in, tiempos, canales, adjacency = cl.matrices_cluster(controles, evocados_co, evocados_in)

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(5, 256, 64)
(5, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(22, 256, 64)
(22, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(31, 256, 64)
(31, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(23, 256, 64)
(23, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delau

In [11]:
controles_dif = controles_co - controles_in
print(np.shape(controles_dif))
exguerrilleros_dif = exguerrilleros_co - exguerrilleros_in
print(np.shape(exguerrilleros_dif))
exparamilitares_dif = exparamilitares_co - exparamilitares_in
print(np.shape(exparamilitares_dif))
victimas_dif = victimas_co - victimas_in
print(np.shape(victimas_dif))
ambos_dif = ambos_co - ambos_in
print(np.shape(ambos_dif))

(16, 256, 64)
(22, 256, 64)
(31, 256, 64)
(23, 256, 64)
(5, 256, 64)


## Análisis ensayos incongruentes

In [13]:
# Análisis de ensayos incongruentes
X = [ambos_in, exguerrilleros_in, exparamilitares_in, victimas_in, controles_in]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.003155 max=7.841937
Running initial clustering
Using 39 thresholds from 0.20 to 7.80 for TFCE computation (h_power=2.00, e_power=0.50)
Found 16384 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [08:45<00:00,    1.90it/s]

Computing cluster p-values


Done.
0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [17]:
df_p_values = cl.p_values_table(p_values, tiempos, canales)
figura = cl.clusters_color(df=df_p_values, cmap='autumn')

In [18]:
# Tabla de clústeres
df_clusteres = cl.lista_clusteres(df_p_values)
df_clusteres

,time_point,electrodes
0,0.578125,"(C1, FCZ)"
1,0.582031,"(C1, FCZ)"
2,0.585938,"(C1, FC2, FCZ, CZ)"
3,0.589844,"(C1, FC2, FCZ, CZ)"
4,0.593750,"(C1, FC2, FCZ, CZ)"
5,0.597656,"(C1, FC2, FCZ, CZ)"
6,0.601562,"(C1, FC2, FCZ, CZ)"
7,0.605469,"(C1, AFZ, FZ, FC2, FCZ, CZ)"
8,0.609375,"(C1, AFZ, FZ, F2, FC2, FCZ, CZ)"
9,0.613281,"(C1, AFZ, FZ, F2, FC2, FCZ, CZ)"


In [19]:
lista, df = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista

FCZ    55.0
C1     54.0
CZ     52.0
T8     41.0
FC3    40.0
       ... 
IZ      0.0
OZ      0.0
FP2     0.0
AF8     0.0
O2      0.0
Length: 64, dtype: float64

In [20]:
cl.time_intervals(df, ['FCZ', 'CZ'])

,cuenta,t_inicial,t_final,delta(ms)
231,18,0.636719,0.703125,66.40625
255,21,0.718750,0.796875,78.12500


## Análisis diferencias ensayos congruentes e incongruentes

In [21]:
# Análisis de diferencias entre ensayos congruentes e incongruentes
X = [ambos_dif, exguerrilleros_dif, exparamilitares_dif, victimas_dif, controles_dif]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.009419 max=8.191951
Running initial clustering
Using 40 thresholds from 0.20 to 8.00 for TFCE computation (h_power=2.00, e_power=0.50)
Found 16384 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [09:13<00:00,    1.81it/s]

Computing cluster p-values


Done.
0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [22]:
df_p_values = cl.p_values_table(p_values, tiempos, canales)
figura = cl.clusters_color(df=df_p_values, cmap='autumn')

In [23]:
# Tabla de clústeres
df_clusteres = cl.lista_clusteres(df_p_values)
df_clusteres

,time_point,electrodes
0,-0.085938,"(FPZ,)"
1,-0.082031,"(FP1, AF3, FPZ, FP2, AF4, AFZ)"
2,-0.078125,"(FP1, AF3, FPZ, FP2, AF4, AFZ, F2, FC2, FCZ)"
3,-0.074219,"(FC1, FPZ, FP2, AF4, AFZ, FC4, FC2, FCZ, CZ)"
4,-0.070312,"(FC1, FC2, FCZ, CZ)"
...,...,...
162,0.781250,"(C1, C3, CP3, CP1, P1, P3, PO3, POZ, PZ, CPZ, ..."
163,0.785156,"(FC1, C1, C3, CP3, CP1, P1, P3, P5, PO3, POZ, ..."
164,0.789062,"(FC1, C1, C3, CP5, CP3, CP1, P1, P3, P5, PO7, ..."
165,0.792969,"(C1, C3, CP5, CP3, CP1, P1, P3, P5, P7, PO3, P..."


In [24]:
lista, df = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista

CZ     132.0
FC1    119.0
F3     113.0
C3     109.0
C1     109.0
       ...  
P8       0.0
P10      0.0
IZ       0.0
OZ       0.0
F8       0.0
Length: 64, dtype: float64

In [25]:
cl.time_intervals(df, ['CZ'])

,cuenta,t_inicial,t_final,delta(ms)
109,17,0.164062,0.226562,62.50000
139,26,0.246094,0.343750,97.65625
211,37,0.484375,0.625000,140.62500
255,21,0.718750,0.796875,78.12500
